## INFO442
## News Article Recommender System
## Group 3 (Kevin Shi, Benjamin Leung, Kathryn Swatek, Richardson Chhin)
## Notebook for Data Preprocessing

## Preprocessing Steps:
**1. Clean behaviors_train.tsv:**
  1. Get correct column types (e.g., convert time column to datetime)
  2. Fill NaNs for users with no history
  3. Convert history string to list
  4. Parse impressions into (news_id, label) pairs: (candidate_news_id, clicked)
  5. Explode impressions column into two columns: 'candidate_news_id' (news_id) and 'clicked' (label 0 or 1)

**2. Clean news_train.tsv:**
  1. Drop incomplete articles from news (i.e., rows with NaN in 'abstract' column)
  2. Remove any references to those dropped incomplete articles from:
    1. The History column in behaviors_train.tsv
    2. The Impressions column in behaviors_train.tsv
    3. Remove rows with no impressions remaining
  3. Check for duplicate rows

**3. Text preprocessing news_train.tsv**
  1. Concatenate the content of the 'title' and 'abstract' columns into a column called 'text'
  2. Clean and Tokenize Text:
    1. Removes all characters from the text that are not lowercase letters or whitespace (punctuation, numbers, special symbols)
    2. Make text lowercase
    3. Split the cleaned text into a list of individual words (tokens)
    4. Lemmatization and stopword removal
    5. Apply the above cleaning and tokenization to 'text' column
  3. For TF-IDF: Join tokens back into text
  4. For Word2Vec: Keep tokenized version, but make a plain list of strings
  5. For BERT: Combine title and abstract as a single string

**4. Checkpoint: saving**
1. After steps 1-3, save as news_train_tokenized.parquet
2. After steps 1-3, save as behaviors_train_preprocessed.parquet

**5. Preprocessing entity_embedding_train.vec and relation_embedding_train.vec**
1. Load embeddings into something like a dictionary, such as: {entity_id: np.array(embedding)}

**6. Item Vector Text Processing (TFIDF, Word2Vec, BERT)**
1. Create a new column to store a vector for each of the following:
  1. One-hot-encode Category and Subcategory to get binary vectors
  2. Get vectors/embeddings for Title and Abstract using TF-IDF
  3. Get vectors/embeddings Title and Abstract using Word2Vec (Gensim)
  4. Get BERT embeddings Title and Abstract using Hugging Face models (use a pretrained BERT tokenizer)

**7. Checkpoint: saving**
1. After step 6, save as news_train_with_embeddings_dense.parquet

**8. Normalization/scaling to all generated vectors (binary, TF-IDF, Word2Vec, BERT) before concatenation**

**9. Concatenate all processed vectors (Category, Subcategory, Title, Abstract) to form final item profile vectors**
1. Because we want to compare the performance of using TFIDF, word2vec, and BERT, we need 3 different representations of item vectors

**10. Checkpoint: saving item vectors**
1. Before saving, make the file size smaller by removing unnecessary columns
2. After steps 8-9, save as news_train_with_item_vectors.parquet
3. *Now we have the item profile*

**11. Build user profile (generate user profiles using user history and item vector averages)**

## Data Dictionary:
This project uses the MIND (Microsoft News Dataset), specifically the `MINDsmall` version. The data is pre-split into training and validation sets.

---
**`behaviors.tsv`**: Contains user click histories and impression logs.

**Columns**: `impression_id`, `user_id`, `time`, `history` (previously clicked news), `impressions` (articles shown to the user and whether they were clicked)

*Renamed to behaviors_train.tsv and behaviors_val.tsv*

---
**`news.tsv`**: Contains information about news articles.

**Columns**: `news_id`, `category`, `subcategory`, `title`, `abstract`, `url`, `title_entities`, `abstract_entities`

*Renamed to news_train.tsv and news_val.tsv*

---
**`entity_embedding.vec` & `relation_embedding.vec`**: Contain 100-dimensional embeddings for entities and relations from a knowledge graph

*Rennamed to entity_embedding_train.vec and relation_embedding_train.vec*

## Imports

In [ ]:
import numpy as np
import pandas as pd

# Data Preprocessing

## Preprocessing behaviors_train.tsv

In [ ]:
# Load behaviors_train.tsv
try:
    behaviors_train = pd.read_csv('MINDsmall_train/behaviors_train.tsv', sep='\t', header=None,
                              names=['impression_id', 'user_id', 'time', 'history', 'impressions'])
except FileNotFoundError:
    from google.colab import files
    files.upload()
    behaviors_train = pd.read_csv('behaviors_train.tsv', sep='\t', header=None,
                              names=['impression_id', 'user_id', 'time', 'history', 'impressions'])

Saving behaviors_train.tsv to behaviors_train.tsv


In [ ]:
behaviors_train.head()

impression_id user_id                   time  \
0              1  U13740  11/11/2019 9:05:58 AM   
1              2  U91836  11/12/2019 6:11:30 PM   
2              3  U73700  11/14/2019 7:01:48 AM   
3              4  U34670  11/11/2019 5:28:05 AM   
4              5   U8125  11/12/2019 4:11:21 PM   

                                             history  \
0  N55189 N42782 N34694 N45794 N18445 N63302 N104...   
1  N31739 N6072 N63045 N23979 N35656 N43353 N8129...   
2  N10732 N25792 N7563 N21087 N41087 N5445 N60384...   
3  N45729 N2203 N871 N53880 N41375 N43142 N33013 ...   
4                        N10078 N56514 N14904 N33740   

                                         impressions  
0                                  N55689-1 N35729-0  
1  N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...  
2  N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...  
3                N35729-0 N33632-0 N49685-1 N27581-0  
4  N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...

In [ ]:
behaviors_train.shape

(156965, 5)

In [ ]:
behaviors_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156965 entries, 0 to 156964
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   impression_id  156965 non-null  int64 
 1   user_id        156965 non-null  object
 2   time           156965 non-null  object
 3   history        153727 non-null  object
 4   impressions    156965 non-null  object
dtypes: int64(1), object(4)
memory usage: 6.0+ MB


In [ ]:
# Convert time to datetime
behaviors_train["time"] = pd.to_datetime(behaviors_train["time"])

In [ ]:
print(behaviors_train.isnull().sum())

impression_id       0
user_id             0
time                0
history          3238
impressions         0
dtype: int64


In [ ]:
# Fill NaNs for users with no history
print("Number of null values in history column:", behaviors_train["history"].isnull().sum())
behaviors_train["history"] = behaviors_train["history"].fillna("")
print("Number of null values in history column after fillna:", behaviors_train["history"].isnull().sum())

Number of null values in history column: 3238
Number of null values in history column after fillna: 0


In [ ]:
# Convert history string to list
behaviors_train["history"] = behaviors_train["history"].apply(lambda x: x.strip().split() if x else [])
print(type(behaviors_train.iloc[0]["history"]))
behaviors_train.head()

<class 'list'>


impression_id user_id                time  \
0              1  U13740 2019-11-11 09:05:58   
1              2  U91836 2019-11-12 18:11:30   
2              3  U73700 2019-11-14 07:01:48   
3              4  U34670 2019-11-11 05:28:05   
4              5   U8125 2019-11-12 16:11:21   

                                             history  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...   
1  [N31739, N6072, N63045, N23979, N35656, N43353...   
2  [N10732, N25792, N7563, N21087, N41087, N5445,...   
3  [N45729, N2203, N871, N53880, N41375, N43142, ...   
4                   [N10078, N56514, N14904, N33740]   

                                         impressions  
0                                  N55689-1 N35729-0  
1  N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...  
2  N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...  
3                N35729-0 N33632-0 N49685-1 N27581-0  
4  N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...

In [ ]:
print(type(behaviors_train.iloc[0]["impressions"]))

<class 'str'>


In [ ]:
# Parse impressions into (news_id, label) pairs
def parse_impressions(imp_str):
    return [(item.split("-")[0], int(item.split("-")[1])) for item in imp_str.split()]

behaviors_train["impressions"] = behaviors_train["impressions"].apply(parse_impressions)
behaviors_train.head()

impression_id user_id                time  \
0              1  U13740 2019-11-11 09:05:58   
1              2  U91836 2019-11-12 18:11:30   
2              3  U73700 2019-11-14 07:01:48   
3              4  U34670 2019-11-11 05:28:05   
4              5   U8125 2019-11-12 16:11:21   

                                             history  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...   
1  [N31739, N6072, N63045, N23979, N35656, N43353...   
2  [N10732, N25792, N7563, N21087, N41087, N5445,...   
3  [N45729, N2203, N871, N53880, N41375, N43142, ...   
4                   [N10078, N56514, N14904, N33740]   

                                         impressions  
0                         [(N55689, 1), (N35729, 0)]  
1  [(N20678, 0), (N39317, 0), (N58114, 0), (N2049...  
2  [(N50014, 0), (N23877, 0), (N35389, 0), (N4971...  
3  [(N35729, 0), (N33632, 0), (N49685, 1), (N2758...  
4  [(N39985, 0), (N36050, 0), (N16096, 0), (N8400...

In [ ]:
print(type(behaviors_train.iloc[0]["impressions"]))       # should be list
print(type(behaviors_train.iloc[0]["impressions"][0]))    # should be tuple
print(type(behaviors_train.iloc[0]["impressions"][0][0])) # should be string
print(type(behaviors_train.iloc[0]["impressions"][0][1])) # should be ?

<class 'list'>
<class 'tuple'>
<class 'str'>
<class 'int'>


## Preprocessing news_train.tsv

In [ ]:
# Load news_train.tsv
try:
    news_train = pd.read_csv('MINDsmall_train/news_train.tsv', sep='\t', header=None,
                   names=['news_id', 'category', 'subcategory', 'title', 'abstract', 'url',
                          'title_entities', 'abstract_entities'], encoding='utf-8')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    news_train = pd.read_csv('news_train.tsv', sep='\t', header=None,
                   names=['news_id', 'category', 'subcategory', 'title', 'abstract', 'url',
                          'title_entities', 'abstract_entities'], encoding='utf-8')

Saving news_train.tsv to news_train.tsv


In [ ]:
news_train.head()

news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                   abstract_entities  
0                                                 []  
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...  
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...  
3  [{"Label": "National Basketball Association", ...  
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...

In [ ]:
print(f'Shape: {news_train.shape}')
print(f'Rows: {len(news_train)}')
print(f'Columns: {len(news_train.columns)}')

Shape: (51282, 8)
Rows: 51282
Columns: 8


In [ ]:
news_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51282 entries, 0 to 51281
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   news_id            51282 non-null  object
 1   category           51282 non-null  object
 2   subcategory        51282 non-null  object
 3   title              51282 non-null  object
 4   abstract           48616 non-null  object
 5   url                51282 non-null  object
 6   title_entities     51279 non-null  object
 7   abstract_entities  51278 non-null  object
dtypes: object(8)
memory usage: 3.1+ MB


### Data Cleaning (handle null values)

In [ ]:
news_train.isnull().sum()

news_id                 0
category                0
subcategory             0
title                   0
abstract             2666
url                     0
title_entities          3
abstract_entities       4
dtype: int64

In [ ]:
news_train[news_train.isna().any(axis=1)].head()

news_id   category        subcategory  \
38   N22028  lifestyle      lifestylebuzz   
41   N41835       news          newsworld   
133  N25174    weather  weathertopstories   
137  N45191  lifestyle           shop-all   
175  N31161     sports       baseball_mlb   

                                                 title abstract  \
38   Mom with schizophrenia, 6-year-old daughter mi...      NaN   
41                        Today in History: November 2      NaN   
133  Winter Storm Warning For Metro Denver, Boulder...      NaN   
137  Amazon Says These Are Its Funniest Customer Re...      NaN   
175  Astros have a lot of baseball left despite two...      NaN   

                                               url  \
38   https://assets.msn.com/labs/mind/AAJfTsZ.html   
41   https://assets.msn.com/labs/mind/AAJwpBS.html   
133  https://assets.msn.com/labs/mind/AAJwpSQ.html   
137  https://assets.msn.com/labs/mind/AAEyYIZ.html   
175  https://assets.msn.com/labs/mind/AAJgPPi.html   

                                        title_entities abstract_entities  
38   [{"Label": "Queens", "Type": "G", "WikidataId"...                []  
41                                                  []                []  
133  [{"Label": "Boulder, Colorado", "Type": "G", "...                []  
137  [{"Label": "Amazon (company)", "Type": "O", "W...                []  
175  [{"Label": "Houston Astros", "Type": "O", "Wik...                []

In [ ]:
print(news_train.apply(lambda row: row.astype(str).isin(['']).any(), axis=1).sum())

0


In [ ]:
print(news_train.apply(lambda row: row.astype(str).isin(['[]']).any(), axis=1).sum())

20874


In [ ]:
print(f"Number of rows in 'title_entities' column containing []: {news_train['title_entities'].apply(lambda x: x == '[]').sum()}")

Number of rows in 'title_entities' column containing []: 13842


In [ ]:
print(f"Number of rows in 'abstract_entities' column containing []: {news_train['abstract_entities'].apply(lambda x: x == '[]').sum()}")

Number of rows in 'abstract_entities' column containing []: 13825


**There are quite a lot of rows with '[]' in the 'title_entities' and 'abstract entities' columns (20874 rows out of 51282 rows). Let's hold off on removing these for now. If we want to leverage the entity embeddings and relation embeddings, then we might need to remove these rows.**

If we drop news articles from news_train.tsv due to null values (e.g., missing abstract, title_entities, or abstract_entities), we must also remove any references to those articles from:

* the History column in behaviors_train.tsv

* the Impressions column in behaviors_train.tsv <br/>

Otherwise, our model may try to build user/item profiles based on missing data, which leads to inconsistencies or errors.

In [ ]:
# Drop incomplete articles from news (i.e., rows with NaN)
news_train_no_nan = news_train.dropna(subset=["abstract", "title_entities", "abstract_entities"])
valid_news_ids = set(news_train_no_nan["news_id"])
news_train_no_nan.shape

(48612, 8)

In [ ]:
behaviors_train[behaviors_train["impressions"].map(len) == 0]

Empty DataFrame
Columns: [impression_id, user_id, time, history, impressions]
Index: []

In [ ]:
# Clean behaviors_train.tsv references
print(behaviors_train.shape)

# Remove invalid news from user history
behaviors_train["history"] = behaviors_train["history"].apply(
    lambda h: [nid for nid in h if nid in valid_news_ids]
)

# Remove invalid impressions
def clean_impressions(imps):
    return [(nid, label) for nid, label in imps if nid in valid_news_ids]

behaviors_train["impressions"] = behaviors_train["impressions"].apply(clean_impressions)

print(behaviors_train.shape)

(156965, 5)
(156965, 5)


In [ ]:
# Check number of rows with no impressions remaining after removing invalid impressions
behaviors_train[behaviors_train["impressions"].map(len) == 0]

impression_id user_id                time  \
30281          30282  U67288 2019-11-09 01:35:20   
42923          42924  U81251 2019-11-12 02:57:53   

                                                 history impressions  
30281  [N1954, N53125, N24045, N37003, N17542, N6831,...          []  
42923  [N50755, N5526, N15029, N32867, N50299, N60607...          []

In [ ]:
# Remove rows with no impressions remaining
behaviors_train = behaviors_train[behaviors_train["impressions"].map(len) > 0]

In [ ]:
# Check number of rows with no impressions remaining after removing rows with no impressions remaining
behaviors_train[behaviors_train["impressions"].map(len) == 0]

Empty DataFrame
Columns: [impression_id, user_id, time, history, impressions]
Index: []

In [ ]:
behaviors_train.shape

(156963, 5)

In [ ]:
'''
cleaned_news_train = news_train.dropna()

cleaned_news_train = cleaned_news_train.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Some rows may contain empty strings or lists, which we also want to remove
mask = ~cleaned_news_train.apply(lambda row: row.astype(str).isin(['', '[]']).any(), axis=1)
cleaned_news_train = cleaned_news_train[mask]
cleaned_news_train.reset_index(drop=True, inplace=True)
'''

"\ncleaned_news_train = news_train.dropna()\n\ncleaned_news_train = cleaned_news_train.applymap(lambda x: x.strip() if isinstance(x, str) else x)\n\n# Some rows may contain empty strings or lists, which we also want to remove\nmask = ~cleaned_news_train.apply(lambda row: row.astype(str).isin(['', '[]']).any(), axis=1)\ncleaned_news_train = cleaned_news_train[mask]\ncleaned_news_train.reset_index(drop=True, inplace=True)\n"

In [ ]:
duplicate_count = news_train.duplicated(subset='news_id').sum()
print(f'Duplicated news_id entries: {duplicate_count}')

Duplicated news_id entries: 0


In [ ]:
news_train_no_nan.head()

news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                   abstract_entities  
0                                                 []  
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...  
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...  
3  [{"Label": "National Basketball Association", ...  
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...

In [ ]:
news_train_no_nan.shape

(48612, 8)

### Text Preprocessing

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Concatenate the content of the 'title' and 'abstract' columns into a column called 'text'
news_train['text'] = news_train['title'] + " " + news_train['abstract']

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
def clean_and_tokenize_text(text):
  if not isinstance(text, str):
      text = str(text)

  # Removes all characters from the text that are not lowercase letters or whitespace (punctuation, numbers, special symbols)
  text = re.sub(r'[^a-z\s]', '', text)

  text = text.lower() # Make text lowercase

  tokens = word_tokenize(text) # Split the cleaned text into a list of individual words (tokens)

  cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

  return cleaned_tokens

In [ ]:
# Apply to 'text' column
news_train['text_tokens'] = news_train['text'].apply(clean_and_tokenize_text)

In [ ]:
# Apply to title and abstract
news_train["title_tokens"] = news_train["title"].apply(clean_and_tokenize_text)
news_train["abstract_tokens"] = news_train["abstract"].apply(clean_and_tokenize_text)

In [ ]:
# For TF-IDF: Join tokens back into text
news_train["text_tfidf"] = news_train["text_tokens"].apply(lambda tokens: " ".join(tokens))

In [ ]:
# For Word2Vec: Keep tokenized version
news_train["text_w2v"] = news_train["text_tokens"]

In [ ]:
# Combine title and abstract as a single string for BERT input
def combine_for_bert(row):
    title = str(row["title"]) #if pd.notnull(row["title"]) else ""
    abstract = str(row["abstract"]) #if pd.notnull(row["abstract"]) else ""
    return title.strip() + " " + abstract.strip()

In [ ]:
# Combine title and abstract as a single string for BERT input
news_train["text_bert"] = news_train.apply(combine_for_bert, axis=1)

In [ ]:
news_train.head(3)

news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   
2  The Cost of Trump's Aid Freeze in the Trenches...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   
2       [ost, rump, id, reeze, renches, kraines, ar]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   
2  [van, olchanets, peeked, parapet, sand, bag, f...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   
2  ost rump id reeze renches kraines ar van olcha...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                           text_bert  
0  The Brands Queen Elizabeth, Prince Charles, an...  
1  50 Worst Habits For Belly Fat These seemingly ...  
2  The Cost of Trump's Aid Freeze in the Trenches...

In [ ]:
news_train[['title', 'abstract', 'text', 'text_tokens', 'title_tokens', 'abstract_tokens', 'text_tfidf', 'text_w2v', 'text_bert']].head(3)

title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   
2  The Cost of Trump's Aid Freeze in the Trenches...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   
2       [ost, rump, id, reeze, renches, kraines, ar]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   
2  [van, olchanets, peeked, parapet, sand, bag, f...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   
2  ost rump id reeze renches kraines ar van olcha...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                           text_bert  
0  The Brands Queen Elizabeth, Prince Charles, an...  
1  50 Worst Habits For Belly Fat These seemingly ...  
2  The Cost of Trump's Aid Freeze in the Trenches...

## Checkpoint: Save news_train

In [ ]:
print(type(news_train))
print(news_train.shape)
news_train.head(2)

<class 'pandas.core.frame.DataFrame'>
(51282, 15)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   

                                           text_bert  
0  The Brands Queen Elizabeth, Prince Charles, an...  
1  50 Worst Habits For Belly Fat These seemingly ...

In [ ]:
# Define the filename for the downloaded parquet
filename = 'news_train_tokenized.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
news_train.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame downloaded as: news_train_tokenized.parquet


## Checkpoint: Explode and save behaviors_train

In [ ]:
behaviors_train_exploded = behaviors_train.explode('impressions').dropna()
behaviors_train_exploded['candidate_news_id'] = behaviors_train_exploded['impressions'].apply(lambda x: x[0])
behaviors_train_exploded['clicked'] = behaviors_train_exploded['impressions'].apply(lambda x: x[1])

In [ ]:
behaviors_train_exploded = behaviors_train_exploded.drop(columns='impressions')

In [ ]:
print("behaviors_train shape:", behaviors_train.shape)
behaviors_train.head()

behaviors_train shape: (156963, 5)


impression_id user_id                time  \
0              1  U13740 2019-11-11 09:05:58   
1              2  U91836 2019-11-12 18:11:30   
2              3  U73700 2019-11-14 07:01:48   
3              4  U34670 2019-11-11 05:28:05   
4              5   U8125 2019-11-12 16:11:21   

                                             history  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...   
2  [N10732, N25792, N7563, N21087, N41087, N5445,...   
3  [N45729, N2203, N871, N53880, N41375, N43142, ...   
4                   [N10078, N56514, N14904, N33740]   

                                         impressions  
0                         [(N55689, 1), (N35729, 0)]  
1  [(N20678, 0), (N39317, 0), (N20495, 0), (N4297...  
2  [(N50014, 0), (N23877, 0), (N35389, 0), (N4971...  
3            [(N35729, 0), (N49685, 1), (N27581, 0)]  
4  [(N39985, 0), (N36050, 0), (N16096, 0), (N8400...

In [ ]:
print("behaviors_train_exploded shape:", behaviors_train_exploded.shape)
behaviors_train_exploded.head()

behaviors_train_exploded shape: (5596624, 6)


impression_id user_id                time  \
0              1  U13740 2019-11-11 09:05:58   
0              1  U13740 2019-11-11 09:05:58   
1              2  U91836 2019-11-12 18:11:30   
1              2  U91836 2019-11-12 18:11:30   
1              2  U91836 2019-11-12 18:11:30   

                                             history candidate_news_id  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...            N55689   
0  [N55189, N42782, N34694, N45794, N18445, N6330...            N35729   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N20678   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N39317   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N20495   

   clicked  
0        1  
0        0  
1        0  
1        0  
1        0

In [ ]:
behaviors_train_exploded.iloc[40:50, :]

impression_id user_id                time  \
2              3  U73700 2019-11-14 07:01:48   
2              3  U73700 2019-11-14 07:01:48   
2              3  U73700 2019-11-14 07:01:48   
2              3  U73700 2019-11-14 07:01:48   
2              3  U73700 2019-11-14 07:01:48   
2              3  U73700 2019-11-14 07:01:48   
3              4  U34670 2019-11-11 05:28:05   
3              4  U34670 2019-11-11 05:28:05   
3              4  U34670 2019-11-11 05:28:05   
4              5   U8125 2019-11-12 16:11:21   

                                             history candidate_news_id  \
2  [N10732, N25792, N7563, N21087, N41087, N5445,...            N50592   
2  [N10732, N25792, N7563, N21087, N41087, N5445,...            N13131   
2  [N10732, N25792, N7563, N21087, N41087, N5445,...             N3839   
2  [N10732, N25792, N7563, N21087, N41087, N5445,...            N12330   
2  [N10732, N25792, N7563, N21087, N41087, N5445,...            N47098   
2  [N10732, N25792, N7563, N21087, N41087, N5445,...            N51570   
3  [N45729, N2203, N871, N53880, N41375, N43142, ...            N35729   
3  [N45729, N2203, N871, N53880, N41375, N43142, ...            N49685   
3  [N45729, N2203, N871, N53880, N41375, N43142, ...            N27581   
4                   [N10078, N56514, N14904, N33740]            N39985   

   clicked  
2        0  
2        0  
2        0  
2        0  
2        0  
2        0  
3        0  
3        1  
3        0  
4        0

In [ ]:
# Define the filename for the downloaded parquet
filename = 'behaviors_train_preprocessed.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
behaviors_train_exploded.to_parquet(filename)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame downloaded as: behaviors_train_preprocessed.parquet


## Make sure we are able to load behaviors_train_preprocessed.parquet

In [ ]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    behaviors_train_preprocessed = pd.read_parquet('MINDsmall_train_processed/behaviors_train_preprocessed.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    behaviors_train_preprocessed = pd.read_parquet('behaviors_train_preprocessed.parquet')

Saving behaviors_train_preprocessed.parquet to behaviors_train_preprocessed (1).parquet


In [ ]:
print(type(behaviors_train_preprocessed))
print(behaviors_train_preprocessed.shape)
behaviors_train_preprocessed.head()

<class 'pandas.core.frame.DataFrame'>
(5596624, 6)


impression_id user_id                time  \
0              1  U13740 2019-11-11 09:05:58   
0              1  U13740 2019-11-11 09:05:58   
1              2  U91836 2019-11-12 18:11:30   
1              2  U91836 2019-11-12 18:11:30   
1              2  U91836 2019-11-12 18:11:30   

                                             history candidate_news_id  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...            N55689   
0  [N55189, N42782, N34694, N45794, N18445, N6330...            N35729   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N20678   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N39317   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N20495   

   clicked  
0        1  
0        0  
1        0  
1        0  
1        0

In [ ]:
'''
behaviors_grouped = behaviors_train_preprocessed.groupby(
    ['impression_id', 'user_id', 'time']
  ).agg({
     'candidate_news_id': list,
     'clicked': list,
     'history': 'first'
  }).reset_index()

behaviors_grouped['impressions'] = behaviors_grouped.apply(
  lambda row: list(zip(row['candidate_news_id'], row['clicked'])),
  axis=1
)
'''

"\nbehaviors_grouped = behaviors_train_preprocessed.groupby(\n    ['impression_id', 'user_id', 'time']\n  ).agg({\n     'candidate_news_id': list,\n     'clicked': list,\n     'history': 'first'\n  }).reset_index()\n\nbehaviors_grouped['impressions'] = behaviors_grouped.apply(\n  lambda row: list(zip(row['candidate_news_id'], row['clicked'])),\n  axis=1\n)\n"

## Preprocessing entity_embedding_train.vec

In [ ]:
# TODO:
# I might be wrong, but I think the last time I took a look at this file,
# the format of the contents in it was entity_id embedding1 embedding2 ... embedding100
# If we are going to use entity embeddings, we need to load it into something like a dictionary? Such as: {entity_id: np.array(embedding)}

import numpy as np
import pandas as pd
from google.colab import files
files.upload()
file_path_entity = 'entity_embedding_train.vec'

entity_embeddings = {}

Saving entity_embedding_train.vec to entity_embedding_train (1).vec


In [ ]:
with open(file_path_entity, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) != 101:
            continue
        entity_id = parts[0]
        embedding = np.array(list(map(float, parts[1:])), dtype=np.float32)
        entity_embeddings[entity_id] = embedding

for entity, vector in list(entity_embeddings.items())[:3]:
    print(f"{entity}: {vector[:5]}...")

print(f"\nLoaded {len(entity_embeddings)} entity embeddings.")
type(entity_embeddings)

Q41: [-0.063388 -0.181451  0.057501 -0.091254 -0.076217]...
Q1860: [ 0.060958  0.069934  0.015832  0.079471 -0.023362]...
Q39631: [-0.093106 -0.052002  0.020556 -0.020801  0.04318 ]...

Loaded 26904 entity embeddings.


dict

## Preprocessing relation_embedding_train.vec

In [ ]:
# TODO:
# Do the same preprocessing as with entity_embedding.vec (try to get a dictionary like {relation: np.array(embedding)})

import numpy as np
import pandas as pd
from google.colab import files
files.upload()
file_path_relation = 'relation_embedding_train.vec'

relation_embeddings = {}

Saving relation_embedding_train.vec to relation_embedding_train.vec


In [ ]:
with open(file_path_relation, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) != 101:
            continue
        relation_id = parts[0]
        embedding = np.array(list(map(float, parts[1:])), dtype=np.float32)
        relation_embeddings[relation_id] = embedding

for relation, vector in list(relation_embeddings.items())[:3]:
    print(f"{relation}: {vector[:5]}...")

print(f"\nLoaded {len(relation_embeddings)} relation embeddings.")
type(relation_embeddings)

P31: [-0.073467 -0.132227  0.034173 -0.032769  0.008289]...
P21: [-0.078436  0.108589 -0.049429 -0.131355  0.0493  ]...
P106: [-0.052137  0.052444 -0.019886 -0.152309  0.014144]...

Loaded 1091 relation embeddings.


dict

# Text Processing

In [1]:
import numpy as np
import pandas as pd

## Load preprocessed news_train_tokenized.parquet

In [2]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    news_train_tokenized = pd.read_parquet('MINDsmall_train_processed/news_train_tokenized.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    news_train_tokenized = pd.read_parquet('news_train_tokenized.parquet')

Saving news_train_tokenized.parquet to news_train_tokenized (1).parquet


In [3]:
print(type(news_train_tokenized))
print(news_train_tokenized.shape)
news_train_tokenized.head()

<class 'pandas.core.frame.DataFrame'>
(51282, 15)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
3  [{"Label": "National Basketball Association", ...   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   
2       [ost, rump, id, reeze, renches, kraines, ar]   
3          [n, ife, eres, ow, ffected, ental, ealth]   
4      [ow, et, id, kin, ag, ccording, ermatologist]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   
2  [van, olchanets, peeked, parapet, sand, bag, f...   
3  [felt, like, fraud, wife, didnt, help, n, fact...   
4  [hey, seem, harmless, there, good, reason, sho...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   
2  ost rump id reeze renches kraines ar van olcha...   
3  n ife eres ow ffected ental ealth felt like fr...   
4  ow et id kin ag ccording ermatologist hey seem...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...   

                                           text_bert  
0  The Brands Que

## One-hot-encode Category and Subcategory to get binary vectors

In [4]:
print(news_train_tokenized['category'].nunique())
news_train_tokenized['category'].value_counts()

17


category
news             15774
sports           14510
finance           3107
foodanddrink      2551
lifestyle         2479
travel            2350
video             2068
weather           2048
health            1885
autos             1639
tv                 889
music              769
movies             606
entertainment      587
kids                17
middleeast           2
northamerica         1
Name: count, dtype: int64

In [5]:
news_train_tokenized['subcategory'].value_counts()

subcategory
newsus                  6564
football_nfl            5420
newspolitics            2826
newscrime               2254
weathertopstories       2047
                        ... 
spendingandborrowing       1
lifestyle-wedding          1
causes-poverty             1
celebritynews              1
watch                      1
Name: count, Length: 264, dtype: int64

In [6]:
# TODO:
category_onehot = pd.get_dummies(news_train_tokenized['category'], prefix='cat')
subcategory_onehot = pd.get_dummies(news_train_tokenized['subcategory'], prefix='subcat')

news_train_tokenized = pd.concat([news_train_tokenized, category_onehot, subcategory_onehot], axis=1)

print("News DataFrame with One-Hot Encoded Category/Subcategory:")
print(news_train_tokenized.shape)

News DataFrame with One-Hot Encoded Category/Subcategory:
(51282, 296)


In [7]:
news_train_tokenized.head(1)

news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   

                                      title_entities abstract_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...                []   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   

                                         text_tokens  ... subcat_voices  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...  ...         False   

  subcat_watch subcat_weatherfullscreenmaps subcat_weathertopstories  \
0        False                        False                    False   

  subcat_weight-loss  subcat_weightloss  subcat_wellness  subcat_wines  \
0              False              False            False         False   

   subcat_wonder  subcat_yearinoffbeatgoodnews  
0          False                         False  

[1 rows x 296 columns]

In [8]:
# Get all column names that start with 'cat_' or 'subcat_'
onehot_cols = [col for col in news_train_tokenized.columns if col.startswith('cat_') or col.startswith('subcat_')]

# Create a column that stores binary vector
news_train_tokenized['vector_cat_subcat'] = news_train_tokenized[onehot_cols].values.tolist()

In [9]:
news_train_tokenized.head(1)

news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   

                                      title_entities abstract_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...                []   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   

                                         text_tokens  ... subcat_watch  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...  ...        False   

  subcat_weatherfullscreenmaps subcat_weathertopstories subcat_weight-loss  \
0                        False                    False              False   

  subcat_weightloss  subcat_wellness  subcat_wines  subcat_wonder  \
0             False            False         False          False   

   subcat_yearinoffbeatgoodnews  \
0                         False   

                                   vector_cat_subcat  
0  [False, False, False, False, False, False, Tru...  

[1 rows x 297 columns]

In [10]:
# Convert booleans to 0/1s
news_train_tokenized['vector_cat_subcat'] = news_train_tokenized['vector_cat_subcat'].apply(
    lambda x: [int(val) for val in x]
)

In [11]:
# Drop the ohe columns
columns_to_drop = [col for col in news_train_tokenized.columns if col.startswith('cat_') or col.startswith('subcat_')]
news_train_tokenized.drop(columns=columns_to_drop, inplace=True)

In [12]:
print(news_train_tokenized.shape)
news_train_tokenized.head(3)

(51282, 16)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   
2  The Cost of Trump's Aid Freeze in the Trenches...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   
2       [ost, rump, id, reeze, renches, kraines, ar]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   
2  [van, olchanets, peeked, parapet, sand, bag, f...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   
2  ost rump id reeze renches kraines ar van olcha...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   
2  The Cost of Trump's Aid Freeze in the Trenches...   

                                   vector_cat_subcat  
0  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...

## Get vectors/embeddings for Title and Abstract using TF-IDF

In [13]:
# TODO: Get vectors/embeddings for Title and Abstract using TF-IDF and store vectors in a new column
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = news_train_tokenized['text_tfidf'].astype(str).tolist()

tfidf_vectorizer = TfidfVectorizer(max_features=5000)

tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

news_train_tokenized['vector_tfidf'] = list(tfidf_matrix)

print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
print(news_train_tokenized[['news_id', 'vector_tfidf']].head(3))

TF-IDF matrix shape: (51282, 5000)
  news_id                                       vector_tfidf
0  N55528    (0, 3620)\t0.2588589414063623\n  (0, 4632)\t...
1  N19639    (0, 3174)\t0.3698965367990924\n  (0, 1393)\t...
2  N61837    (0, 3191)\t0.20505382251567944\n  (0, 3943)\...


In [14]:
# IMPORTANT: SAVE TFIDF MODEL!!
import joblib
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [15]:
print(tfidf_vectorizer.get_feature_names_out()[:5])

['ab' 'abandoned' 'abbard' 'abe' 'ability']


In [16]:
print(f"Train vocabulary size: {len(tfidf_vectorizer.vocabulary_)}")
print(f"Train vocabulary example (first 5): {list(tfidf_vectorizer.vocabulary_.items())[:5]}")
print(f"Train IDF example (first 5): {tfidf_vectorizer.idf_[:5]}")

Train vocabulary size: 5000
Train vocabulary example (first 5): [('rand', 3620), ('ueen', 4632), ('lizabeth', 2598), ('rince', 3852), ('harles', 1959)]
Train IDF example (first 5): [8.06092496 7.43839534 8.23419668 7.89387087 7.26014711]


In [17]:
files.download('tfidf_vectorizer.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
tfidf_matrix

<51282x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 934339 stored elements in Compressed Sparse Row format>

In [19]:
news_train_tokenized.head(2)

news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   

                                   vector_cat_subcat  \
0  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                        vector_tfidf  
0    (0, 3620)\t0.2588589414063623\n  (0, 4632)\t...  
1    (0, 3174)\t0.3698965367990924\n  (0, 1393)\t...

In [20]:
print(news_train_tokenized.shape)
news_train_tokenized['vector_tfidf'][0].toarray()

(51282, 17)


array([[0., 0., 0., ..., 0., 0., 0.]])

## Get vectors/embeddings Title and Abstract using Word2Vec (Gensim)

In [28]:
pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatib

In [21]:
type(news_train_tokenized['text_w2v'].tolist())

list

In [22]:
# debug Return True if any token is not a plain str
def is_bad(doc):
    if not isinstance(doc, list):
        return True
    for tok in doc:
        if not isinstance(tok, str):
            return True
    return False

bad_rows = news_train_tokenized["text_w2v"].apply(is_bad)
print(f"Bad rows: {bad_rows.sum()} / {len(news_train_tokenized)}")

Bad rows: 51282 / 51282


In [23]:
idx = bad_rows.idxmax()      # first bad row
print(news_train_tokenized.loc[idx, "text_w2v"])

['rand' 'ueen' 'lizabeth' 'rince' 'harles' 'rince' 'hilip' 'wear' 'hop'
 'notebook' 'jacket' 'royal' 'cant' 'live' 'without']


In [24]:
def sanitize(doc):
    # make sure we have a list
    if isinstance(doc, np.ndarray):
        doc = doc.tolist()
    elif not isinstance(doc, list):
        doc = list(doc) if doc is not None else []

    # keep only string tokens
    return [str(tok) for tok in doc if isinstance(tok, str)]

news_train_tokenized["text_w2v"] = news_train_tokenized["text_w2v"].apply(sanitize)

In [25]:
bad_rows = news_train_tokenized["text_w2v"].apply(is_bad)
print(f"Bad rows: {bad_rows.sum()} / {len(news_train_tokenized)}")

Bad rows: 0 / 51282


In [26]:
print(news_train_tokenized.loc[0, "text_w2v"])

['rand', 'ueen', 'lizabeth', 'rince', 'harles', 'rince', 'hilip', 'wear', 'hop', 'notebook', 'jacket', 'royal', 'cant', 'live', 'without']


In [27]:
# TODO: Get vectors/embeddings Title and Abstract using Word2Vec (Gensim) and store vectors in a new column
import gensim
from gensim.models import Word2Vec

sentences = news_train_tokenized['text_w2v'].tolist()

w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4, sg=1, seed=42)

def get_w2v_vector(tokens, model):
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

news_train_tokenized['vector_w2v'] = news_train_tokenized['text_tokens'].apply(lambda tokens: get_w2v_vector(tokens, w2v_model))

print(news_train_tokenized[['news_id', 'vector_w2v']].head(3))

  news_id                                         vector_w2v
0  N55528  [0.047399126, -0.09589149, -0.13837357, 0.0836...
1  N19639  [0.09265002, -0.10189674, -0.05178696, 0.04061...
2  N61837  [0.030284544, -0.030569091, -0.072350785, 0.10...


Above code cell took 42 seconds to run

In [28]:
# IMPORTANT: SAVE WORD2VEC MODEL!!
w2v_model.save('word2vec.model')

In [30]:
files.download('word2vec.model')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
print(news_train_tokenized.shape)
news_train_tokenized.head(2)

(51282, 18)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   

                                   vector_cat_subcat  \
0  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                        vector_tfidf  \
0    (0, 3620)\t0.2588589414063623\n  (0, 4632)\t...   
1    (0, 3174)\t0.3698965367990924\n  (0, 1393)\t...   

                                          vector_w2v  
0  [0.047399126, -0.09589149, -0.13837357, 0.0836...  
1  [0.09265002, -0.10189674, -0.05178696, 0.04061...

## Get BERT embeddings Title and Abstract using Hugging Face models (use a pretrained BERT tokenizer)

In [32]:
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

In [33]:
# TODO: Get BERT embeddings Title and Abstract using Hugging Face models and store vectors in a new column
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze().cpu().numpy()

bert_vectors = []
for text in tqdm(news_train_tokenized['text_bert'].astype(str).tolist(), desc="Encoding with BERT"):
    vec = get_bert_embedding(text)
    bert_vectors.append(vec)

news_train_tokenized['vector_bert'] = bert_vectors
print(news_train_tokenized[['news_id', 'vector_bert']].head(2))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Encoding with BERT: 100%|██████████| 51282/51282 [08:32<00:00, 99.98it/s] 


  news_id                                        vector_bert
0  N55528  [0.03229502, 0.50512636, 0.13134134, 0.2930624...
1  N19639  [0.5096505, 0.102168426, 0.40285626, 0.3637481...


In [ ]:
# IMPORTANT: SAVE BERT MODEL!!
model.save_pretrained('./bert-base')
tokenizer.save_pretrained('./bert-base')

Above code cell took 8 minutes to run with colab's GPU. Without colab's GPU, it would have taken much longer (for example, after 9 minutes, it was only at 5%).

In [34]:
print(news_train_tokenized.shape)
news_train_tokenized.head()

(51282, 19)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
3  [{"Label": "National Basketball Association", ...   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   
2       [ost, rump, id, reeze, renches, kraines, ar]   
3          [n, ife, eres, ow, ffected, ental, ealth]   
4      [ow, et, id, kin, ag, ccording, ermatologist]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   
2  [van, olchanets, peeked, parapet, sand, bag, f...   
3  [felt, like, fraud, wife, didnt, help, n, fact...   
4  [hey, seem, harmless, there, good, reason, sho...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   
2  ost rump id reeze renches kraines ar van olcha...   
3  n ife eres ow ffected ental ealth felt like fr...   
4  ow et id kin ag ccording ermatologist hey seem...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...   

                                           text_bert  \
0  The Brands Qu

In [35]:
print(len(news_train_tokenized['vector_bert'][0]))
news_train_tokenized['vector_bert'][0]

768


array([ 3.22950184e-02,  5.05126357e-01,  1.31341338e-01,  2.93062478e-01,
       -5.57363451e-01, -3.66113633e-01,  1.62348747e-01,  5.57718277e-01,
       -3.74193907e-01, -3.54691714e-01, -1.38011441e-01,  1.48088947e-01,
        1.45582244e-01,  1.00607872e+00,  4.61494982e-01,  1.34784982e-01,
       -3.11519563e-01,  9.27519858e-01,  6.10260129e-01, -6.68620542e-02,
       -8.61717761e-02,  7.46763870e-02, -1.22379325e-01,  4.50138360e-01,
        5.93311489e-01,  3.04145575e-01,  1.15371764e-01, -4.33331460e-01,
       -2.23647788e-01,  4.67055351e-01, -3.27056020e-01,  3.03984255e-01,
       -3.07067186e-01, -5.61102986e-01,  3.99101406e-01, -3.37750971e-01,
        4.60502326e-01, -4.23770368e-01, -1.44565582e-01, -3.72148752e-01,
        1.32570267e-01,  1.04441740e-01,  2.88870186e-01, -1.97250903e-01,
       -1.16919927e-01, -5.91276646e-01, -3.29417801e+00,  3.00590485e-01,
        2.05668792e-01, -2.87197560e-01,  1.68726116e-01, -7.83745766e-01,
        9.87700075e-02,  

## Checkpoint: Save news_train_tokenized

In [36]:
print(type(news_train_tokenized))
print(news_train_tokenized.shape)
news_train_tokenized.head(2)

<class 'pandas.core.frame.DataFrame'>
(51282, 19)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   

                                   vector_cat_subcat  \
0  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                        vector_tfidf  \
0    (0, 3620)\t0.2588589414063623\n  (0, 4632)\t...   
1    (0, 3174)\t0.3698965367990924\n  (0, 1393)\t...   

                                          vector_w2v  \
0  [0.047399126, -0.09589149, -0.13837357, 0.0836...   
1  [0.09265002, -0.10189674, -0.05178696, 0.04061...   

                                         vector_bert  
0  [0.03229502, 0.50512636, 0.13134134, 0.2930624...  
1  [0.5096505, 0.102168426, 0.40285626, 0.3637481...

In [37]:
print(news_train_tokenized['vector_tfidf'][0])
news_train_tokenized['vector_tfidf'][0].toarray()

  (0, 3620)	0.2588589414063623
  (0, 4632)	0.2896856259993324
  (0, 2598)	0.25706190475590734
  (0, 3852)	0.5466155134989971
  (0, 1959)	0.2763176301512922
  (0, 4864)	0.29475349174752946
  (0, 2084)	0.2980356815605162
  (0, 3927)	0.28277919343931823
  (0, 726)	0.2356566551182428
  (0, 2594)	0.2244142970661041
  (0, 4920)	0.21514812312551826


array([[0., 0., 0., ..., 0., 0., 0.]])

In [38]:
news_train_tokenized['vector_tfidf_dense'] = news_train_tokenized['vector_tfidf'].apply(
    lambda x: x.toarray().flatten() # Convert to dense and flatten to 1D array
)

In [39]:
# Drop the original sparse column
news_train_tokenized_to_save = news_train_tokenized.drop(columns=['vector_tfidf'])

In [40]:
print(type(news_train_tokenized_to_save))
print(news_train_tokenized_to_save.shape)
news_train_tokenized_to_save.head(2)

<class 'pandas.core.frame.DataFrame'>
(51282, 19)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   

                                   vector_cat_subcat  \
0  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          vector_w2v  \
0  [0.047399126, -0.09589149, -0.13837357, 0.0836...   
1  [0.09265002, -0.10189674, -0.05178696, 0.04061...   

                                         vector_bert  \
0  [0.03229502, 0.50512636, 0.13134134, 0.2930624...   
1  [0.5096505, 0.102168426, 0.40285626, 0.3637481...   

                                  vector_tfidf_dense  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

In [41]:
# Define the filename for the downloaded parquet
filename = 'news_train_with_embeddings_dense.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
news_train_tokenized_to_save.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame downloaded as: news_train_with_embeddings_dense.parquet


## Load news_train_with_embeddings_dense.parquet

In [1]:
import numpy as np
import pandas as pd

In [2]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    news_train_tokenized_to_save = pd.read_parquet('MINDsmall_train_processed/news_train_with_embeddings_dense.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    news_train_tokenized_to_save = pd.read_parquet('news_train_with_embeddings_dense.parquet')

Saving news_train_with_embeddings_dense.parquet to news_train_with_embeddings_dense.parquet


Above cell took 18 minutes to load!

In [3]:
print(type(news_train_tokenized_to_save))
print(news_train_tokenized_to_save.shape)
news_train_tokenized_to_save.head()

<class 'pandas.core.frame.DataFrame'>
(51282, 19)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
3  [{"Label": "National Basketball Association", ...   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   
2       [ost, rump, id, reeze, renches, kraines, ar]   
3          [n, ife, eres, ow, ffected, ental, ealth]   
4      [ow, et, id, kin, ag, ccording, ermatologist]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   
2  [van, olchanets, peeked, parapet, sand, bag, f...   
3  [felt, like, fraud, wife, didnt, help, n, fact...   
4  [hey, seem, harmless, there, good, reason, sho...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   
2  ost rump id reeze renches kraines ar van olcha...   
3  n ife eres ow ffected ental ealth felt like fr...   
4  ow et id kin ag ccording ermatologist hey seem...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...   

                                           text_bert  \
0  The Brands Qu

## Define a function for normalization/scaling to all generated vectors (binary, TF-IDF, Word2Vec, BERT) before concatenation

In [4]:
# TODO:
def normalize_vector(v):
    v = np.array(v)
    norm = np.linalg.norm(v)
    return v / norm if norm > 0 else v

## Concatenate all processed vectors (Category, Subcategory, Title, Abstract) to form final item profile vectors. Because we want to compare the performance of using TFIDF, word2vec, and BERT, we would need 3 different representations of item vectors:

In [5]:
# For each row, normalize and concatenate
def concat_tfidf(row):
    return np.concatenate([
        normalize_vector(row["vector_cat_subcat"]),
        normalize_vector(row["vector_tfidf_dense"])
    ])

def concat_w2v(row):
    return np.concatenate([
        normalize_vector(row["vector_cat_subcat"]),
        normalize_vector(row["vector_w2v"])
    ])

def concat_bert(row):
    return np.concatenate([
        normalize_vector(row["vector_cat_subcat"]),
        normalize_vector(row["vector_bert"])
    ])

In [6]:
# TODO: store the final item profile vector in a new column
news_train_tokenized_to_save["item_vector_tfidf_dense"] = news_train_tokenized_to_save.apply(concat_tfidf, axis=1)
news_train_tokenized_to_save["item_vector_w2v"]   = news_train_tokenized_to_save.apply(concat_w2v, axis=1)
news_train_tokenized_to_save["item_vector_bert"]  = news_train_tokenized_to_save.apply(concat_bert, axis=1)

In [7]:
print(type(news_train_tokenized_to_save))
print(news_train_tokenized_to_save.shape)
news_train_tokenized_to_save.head()

<class 'pandas.core.frame.DataFrame'>
(51282, 22)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
3  [{"Label": "National Basketball Association", ...   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                         text_tokens  ...  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...  ...   
1  [orst, abits, elly, hese, seemingly, harmless,...  ...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...  ...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...  ...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...  ...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   
2  ost rump id reeze renches kraines ar van olcha...   
3  n ife eres ow ffected ental ealth felt like fr...   
4  ow et id kin ag ccording ermatologist hey seem...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                   vector_cat_subcat  \
0  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...   
3  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                          

In [8]:
print(len(news_train_tokenized_to_save['item_vector_tfidf_dense'][0]))
news_train_tokenized_to_save['item_vector_tfidf_dense'][0]

5281


array([0., 0., 0., ..., 0., 0., 0.])

In [9]:
print(len(news_train_tokenized_to_save['item_vector_w2v'][0]))
news_train_tokenized_to_save['item_vector_w2v'][0]

381


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.70710678,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [10]:
print(len(news_train_tokenized_to_save['item_vector_bert'][0]))
news_train_tokenized_to_save['item_vector_bert'][0]

1049


array([ 0.        ,  0.        ,  0.        , ..., -0.03417296,
        0.03808109,  0.02280931])

## Clean news_train_tokenized_to_save by making the file smaller (get rid of unnecessary columns)

In [11]:
# TODO: remove columns 'text', 'text_tokens', 'title_tokens', 'abstract_tokens', 'text_tfidf', 'text_w2v', 'text_bert',	'vector_cat_subcat', 'vector_w2v', 'vector_bert', 'vector_tfidf_dense'
remove_columns = [
    'text',
    'text_tokens',
    'title_tokens',
    'abstract_tokens',
    'text_tfidf',
    'text_w2v',
    'text_bert',
    'vector_cat_subcat',
    'vector_w2v',
    'vector_bert',
    'vector_tfidf_dense'
]
news_train_tokenized_to_save_final = news_train_tokenized_to_save.drop(columns=remove_columns)

In [12]:
print(type(news_train_tokenized_to_save_final))
print(news_train_tokenized_to_save_final.shape)
news_train_tokenized_to_save_final.head()

<class 'pandas.core.frame.DataFrame'>
(51282, 11)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
3  [{"Label": "National Basketball Association", ...   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                             item_vector_tfidf_dense  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...   
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   

                                     item_vector_w2v  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...   
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   

                                    item_vector_bert  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...  
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...

In [13]:
print(len(news_train_tokenized_to_save_final['item_vector_tfidf_dense'][0]))
news_train_tokenized_to_save_final['item_vector_tfidf_dense'][0]

5281


array([0., 0., 0., ..., 0., 0., 0.])

In [14]:
print(len(news_train_tokenized_to_save_final['item_vector_w2v'][0]))
news_train_tokenized_to_save_final['item_vector_w2v'][0]

381


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.70710678,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [15]:
print(len(news_train_tokenized_to_save_final['item_vector_bert'][0]))
news_train_tokenized_to_save_final['item_vector_bert'][0]

1049


array([ 0.        ,  0.        ,  0.        , ..., -0.03417296,
        0.03808109,  0.02280931])

## Checkpoint: Save item vectors

In [16]:
# Define the filename for the downloaded parquet
filename = 'news_train_with_item_vectors.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
news_train_tokenized_to_save_final.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame downloaded as: news_train_with_item_vectors.parquet


#### Attempt to download as .csv

In [ ]:
import pandas as pd

In [ ]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    news_train = pd.read_parquet('MINDsmall_train_processed/news_train_with_item_vectors.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    news_train = pd.read_parquet('news_train_with_item_vectors.parquet')

Saving news_train_with_item_vectors.parquet to news_train_with_item_vectors.parquet


In [ ]:
print(type(news_train))
print(news_train.shape)
news_train.head()

<class 'pandas.core.frame.DataFrame'>
(51282, 11)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
3  [{"Label": "National Basketball Association", ...   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                             item_vector_tfidf_dense  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...   
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   

                                     item_vector_w2v  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...   
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   

                                    item_vector_bert  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...  
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...

In [ ]:
# Define the filename for the downloaded csv
filename = 'news_train_with_item_vectors.csv'

# Save the DataFrame to a temporary csv file in Colab's environment
news_train.to_csv(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame downloaded as: news_train_with_item_vectors.csv


## Build User Profile

In [1]:
import numpy as np
import pandas as pd

In [2]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    behaviors_train_preprocessed = pd.read_parquet('MINDsmall_train_processed/behaviors_train_preprocessed.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    behaviors_train_preprocessed = pd.read_parquet('behaviors_train_preprocessed.parquet')

Saving behaviors_train_preprocessed.parquet to behaviors_train_preprocessed.parquet


In [3]:
print(type(behaviors_train_preprocessed))
print(behaviors_train_preprocessed.shape)
behaviors_train_preprocessed.head()

<class 'pandas.core.frame.DataFrame'>
(5596624, 6)


impression_id user_id                time  \
0              1  U13740 2019-11-11 09:05:58   
0              1  U13740 2019-11-11 09:05:58   
1              2  U91836 2019-11-12 18:11:30   
1              2  U91836 2019-11-12 18:11:30   
1              2  U91836 2019-11-12 18:11:30   

                                             history candidate_news_id  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...            N55689   
0  [N55189, N42782, N34694, N45794, N18445, N6330...            N35729   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N20678   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N39317   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N20495   

   clicked  
0        1  
0        0  
1        0  
1        0  
1        0

In [4]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    item_profile = pd.read_parquet('MINDsmall_train_processed/news_train_with_item_vectors.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    item_profile = pd.read_parquet('news_train_with_item_vectors.parquet')

Saving news_train_with_item_vectors.parquet to news_train_with_item_vectors.parquet


In [5]:
print(type(item_profile))
print(item_profile.shape)
item_profile.head()

<class 'pandas.core.frame.DataFrame'>
(51282, 11)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
3  [{"Label": "National Basketball Association", ...   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                             item_vector_tfidf_dense  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...   
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   

                                     item_vector_w2v  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...   
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   

                                    item_vector_bert  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...  
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...

In [6]:
# Unexplode behaviors_train_preprocessed
behaviors_train_preprocessed = behaviors_train_preprocessed.groupby(
    ['impression_id', 'user_id', 'time']
  ).agg({
     'candidate_news_id': list,
     'clicked': list,
     'history': 'first'
  }).reset_index()

behaviors_train_preprocessed['impressions'] = behaviors_train_preprocessed.apply(
  lambda row: list(zip(row['candidate_news_id'], row['clicked'])),
  axis=1
)

In [7]:
print(behaviors_train_preprocessed.shape)
behaviors_train_preprocessed.head()

(156963, 7)


impression_id user_id                time  \
0              1  U13740 2019-11-11 09:05:58   
1              2  U91836 2019-11-12 18:11:30   
2              3  U73700 2019-11-14 07:01:48   
3              4  U34670 2019-11-11 05:28:05   
4              5   U8125 2019-11-12 16:11:21   

                                   candidate_news_id  \
0                                   [N55689, N35729]   
1  [N20678, N39317, N20495, N42977, N22407, N1459...   
2  [N50014, N23877, N35389, N49712, N16844, N2381...   
3                           [N35729, N49685, N27581]   
4  [N39985, N36050, N16096, N8400, N22407, N60408...   

                                             clicked  \
0                                             [1, 0]   
1                     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]   
2  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3                                          [0, 1, 0]   
4  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                             history  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...   
2  [N10732, N25792, N7563, N21087, N41087, N5445,...   
3  [N45729, N2203, N871, N53880, N41375, N43142, ...   
4                   [N10078, N56514, N14904, N33740]   

                                         impressions  
0                         [(N55689, 1), (N35729, 0)]  
1  [(N20678, 0), (N39317, 0), (N20495, 0), (N4297...  
2  [(N50014, 0), (N23877, 0), (N35389, 0), (N4971...  
3            [(N35729, 0), (N49685, 1), (N27581, 0)]  
4  [(N39985, 0), (N36050, 0), (N16096, 0), (N8400...

In [8]:
# Create dictionaries for quick lookup
tfidf_dict = dict(zip(item_profile["news_id"], item_profile["item_vector_tfidf_dense"]))
w2v_dict   = dict(zip(item_profile["news_id"], item_profile["item_vector_w2v"]))
bert_dict  = dict(zip(item_profile["news_id"], item_profile["item_vector_bert"]))

In [9]:
# Define Averaging function
def average_vectors(news_ids, item_vector_dict):
    vectors = [item_vector_dict[nid] for nid in news_ids if nid in item_vector_dict]
    return np.mean(vectors, axis=0) if vectors else np.zeros(len(next(iter(item_vector_dict.values()))))

In [10]:
# Get user -> history mapping
user_histories = (
    behaviors_train_preprocessed.groupby("user_id")["history"]
    .first()
    .to_dict()
)

In [11]:
# Build user profile vectors (one per user)
user_profiles_tfidf = {}
user_profiles_w2v = {}
user_profiles_bert = {}

for user_id, history in user_histories.items():
    user_profiles_tfidf[user_id] = average_vectors(history, tfidf_dict)
    user_profiles_w2v[user_id]   = average_vectors(history, w2v_dict)
    user_profiles_bert[user_id]  = average_vectors(history, bert_dict)

In [12]:
# Create a user profiles DataFrame
user_profile_df = pd.DataFrame({
    "user_id": list(user_profiles_tfidf.keys()),
    "user_profile_vector_tfidf_avg": list(user_profiles_tfidf.values()),
    "user_profile_vector_w2v_avg": list(user_profiles_w2v.values()),
    "user_profile_vector_bert_avg": list(user_profiles_bert.values())
})

In [13]:
# Merge into behaviors_train_preprocessed
behaviors_train_preprocessed = behaviors_train_preprocessed.merge(
    user_profile_df, on="user_id", how="left"
)

In [14]:
print(behaviors_train_preprocessed.shape)
behaviors_train_preprocessed.head()

(156963, 10)


impression_id user_id                time  \
0              1  U13740 2019-11-11 09:05:58   
1              2  U91836 2019-11-12 18:11:30   
2              3  U73700 2019-11-14 07:01:48   
3              4  U34670 2019-11-11 05:28:05   
4              5   U8125 2019-11-12 16:11:21   

                                   candidate_news_id  \
0                                   [N55689, N35729]   
1  [N20678, N39317, N20495, N42977, N22407, N1459...   
2  [N50014, N23877, N35389, N49712, N16844, N2381...   
3                           [N35729, N49685, N27581]   
4  [N39985, N36050, N16096, N8400, N22407, N60408...   

                                             clicked  \
0                                             [1, 0]   
1                     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]   
2  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3                                          [0, 1, 0]   
4  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                             history  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...   
2  [N10732, N25792, N7563, N21087, N41087, N5445,...   
3  [N45729, N2203, N871, N53880, N41375, N43142, ...   
4                   [N10078, N56514, N14904, N33740]   

                                         impressions  \
0                         [(N55689, 1), (N35729, 0)]   
1  [(N20678, 0), (N39317, 0), (N20495, 0), (N4297...   
2  [(N50014, 0), (N23877, 0), (N35389, 0), (N4971...   
3            [(N35729, 0), (N49685, 1), (N27581, 0)]   
4  [(N39985, 0), (N36050, 0), (N16096, 0), (N8400...   

                       user_profile_vector_tfidf_avg  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...   
1  [0.0, 0.0, 0.05276916277511548, 0.031661497665...   
2  [0.044194173824159216, 0.0, 0.0441941738241592...   
3  [0.0, 0.0, 0.1414213562373095, 0.0, 0.0, 0.0, ...   
4  [0.17677669529663687, 0.0, 0.0, 0.0, 0.1767766...   

                         user_profile_vector_w2v_avg  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...   
1  [0.0, 0.0, 0.05276916277511548, 0.031661497665...   
2  [0.044194173824159216, 0.0, 0.0441941738241592...   
3  [0.0, 0.0, 0.1414213562373095, 0.0, 0.0, 0.0, ...   
4  [0.17677669529663687, 0.0, 0.0, 0.0, 0.1767766...   

                        user_profile_vector_bert_avg  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...  
1  [0.0, 0.0, 0.05276916277511548, 0.031661497665...  
2  [0.044194173824159216, 0.0, 0.0441941738241592...  
3  [0.0, 0.0, 0.1414213562373095, 0.0, 0.0, 0.0, ...  
4  [0.17677669529663687, 0.0, 0.0, 0.0, 0.1767766...

In [15]:
behaviors_train_preprocessed['user_profile_vector_tfidf_avg'][0]

array([0., 0., 0., ..., 0., 0., 0.])

In [16]:
behaviors_train_preprocessed['user_profile_vector_w2v_avg'][0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.07856742,  0.        ,  0.07856742,  0.        ,
        0.23570226,  0.        ,  0.15713484,  0.        ,  0.15713484,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.07856742,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [17]:
behaviors_train_preprocessed['user_profile_vector_bert_avg'][0]

array([ 0.        ,  0.        ,  0.        , ..., -0.01304343,
        0.04107342,  0.02201502])

## Checkpoint: save user profile

In [18]:
# 1. DataFrame with tf-idf vector
df_tfidf = behaviors_train_preprocessed[[
    'impression_id', 'user_id', 'time', 'candidate_news_id',
    'clicked', 'history', 'impressions', 'user_profile_vector_tfidf_avg'
]]

# 2. DataFrame with word2vec vector
df_w2v = behaviors_train_preprocessed[[
    'impression_id', 'user_id', 'time', 'candidate_news_id',
    'clicked', 'history', 'impressions', 'user_profile_vector_w2v_avg'
]]

# 3. DataFrame with BERT vector
df_bert = behaviors_train_preprocessed[[
    'impression_id', 'user_id', 'time', 'candidate_news_id',
    'clicked', 'history', 'impressions', 'user_profile_vector_bert_avg'
]]

In [19]:
print(df_tfidf.shape)
df_tfidf.head(1)

(156963, 8)


impression_id user_id                time candidate_news_id clicked  \
0              1  U13740 2019-11-11 09:05:58  [N55689, N35729]  [1, 0]   

                                             history  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...   

                  impressions  \
0  [(N55689, 1), (N35729, 0)]   

                       user_profile_vector_tfidf_avg  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...

In [20]:
print(df_w2v.shape)
df_w2v.head(1)

(156963, 8)


impression_id user_id                time candidate_news_id clicked  \
0              1  U13740 2019-11-11 09:05:58  [N55689, N35729]  [1, 0]   

                                             history  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...   

                  impressions  \
0  [(N55689, 1), (N35729, 0)]   

                         user_profile_vector_w2v_avg  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...

In [21]:
print(df_bert.shape)
df_bert.head(1)

(156963, 8)


impression_id user_id                time candidate_news_id clicked  \
0              1  U13740 2019-11-11 09:05:58  [N55689, N35729]  [1, 0]   

                                             history  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...   

                  impressions  \
0  [(N55689, 1), (N35729, 0)]   

                        user_profile_vector_bert_avg  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...

In [22]:
df_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156963 entries, 0 to 156962
Data columns (total 8 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   impression_id                  156963 non-null  int64         
 1   user_id                        156963 non-null  object        
 2   time                           156963 non-null  datetime64[ns]
 3   candidate_news_id              156963 non-null  object        
 4   clicked                        156963 non-null  object        
 5   history                        156963 non-null  object        
 6   impressions                    156963 non-null  object        
 7   user_profile_vector_tfidf_avg  156963 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 9.6+ MB


In [23]:
print(len(df_tfidf["user_profile_vector_tfidf_avg"][0]))
print(df_tfidf["user_profile_vector_tfidf_avg"].iloc[0])
print(type(df_tfidf["user_profile_vector_tfidf_avg"].iloc[0]))
print(type(df_tfidf["user_profile_vector_tfidf_avg"].iloc[0][0]))

5281
[0. 0. 0. ... 0. 0. 0.]
<class 'numpy.ndarray'>
<class 'numpy.float64'>


In [ ]:
# Define the filename for the downloaded parquet
filename = 'behaviors_train_user_profile_tfidf.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
df_tfidf.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

In [ ]:
# Define the filename for the downloaded parquet
filename = 'behaviors_train_user_profile_w2v.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
df_w2v.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

In [ ]:
# Define the filename for the downloaded parquet
filename = 'behaviors_train_user_profile_bert.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
df_bert.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

Save as csv below (not preferable)

In [ ]:
# @title Save as csv below (not preferable)
# Define the filename for the downloaded csv
filename = 'behaviors_train_user_profile.csv'

# Save the DataFrame to a temporary csv file in Colab's environment
behaviors_train_preprocessed.to_csv(filename)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame downloaded as: behaviors_train_user_profile.csv


In [ ]:
behaviors_train_preprocessed['user_id'].nunique()

50000

In [ ]:
behaviors_train_preprocessed[behaviors_train_preprocessed['user_id'] == 'U13740']

impression_id user_id                time  \
0                   1  U13740 2019-11-11 09:05:58   
35261           35263  U13740 2019-11-09 05:59:43   
154834         154837  U13740 2019-11-13 15:27:40   

                                        candidate_news_id  \
0                                        [N55689, N35729]   
35261   [N20020, N3737, N43202, N18708, N30125, N349, ...   
154834  [N13907, N8509, N47061, N22417, N35273, N33831...   

                                                  clicked  \
0                                                  [1, 0]   
35261   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
154834  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...   

                                                  history  \
0       [N55189, N42782, N34694, N45794, N18445, N6330...   
35261   [N55189, N42782, N34694, N45794, N18445, N6330...   
154834  [N55189, N42782, N34694, N45794, N18445, N6330...   

                                              impressions  \
0                              [(N55689, 1), (N35729, 0)]   
35261   [(N20020, 0), (N3737, 0), (N43202, 0), (N18708...   
154834  [(N13907, 0), (N8509, 0), (N47061, 0), (N22417...   

                            user_profile_vector_tfidf_avg  \
0       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...   
35261   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...   
154834  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...   

                              user_profile_vector_w2v_avg  \
0       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...   
35261   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...   
154834  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...   

                             user_profile_vector_bert_avg  
0       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...  
35261   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...  
154834  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0785674201318...